# Step 1.0 Setup

We define the various basic assets, which are used in the demo. These assets are:
 - Database
 - Stages (internal)

It is expected that the snowflake role has the following grants:
 - Ability to create database/schema/stage
  
### Query
Q: Can I use an existing database/schema for executing this demo ?

    A: Yes, in such cases you can skip this notebook. Ensure to update
        the various configurations in the file 'config.ini'

In [1]:
from IPython.display import display, HTML, Image , Markdown
from snowflake.snowpark.session import Session
# from snowflake.snowpark.types import * 
# from snowflake.snowpark.functions import *
import os ,configparser ,json

PROJECT_HOME_DIR = '../../..'
CONFIG_FL = f'{PROJECT_HOME_DIR}/config.ini'

# Source various helper functions
%run ./scripts/notebook_helpers.py

In [2]:
display(Markdown("### Initialization"))

config = configparser.ConfigParser()
sflk_session = None

print(" Initialize Snowpark session")
with open(CONFIG_FL) as f:
    config.read(CONFIG_FL)
    snow_conn_flpath =  f"{PROJECT_HOME_DIR}/{config['DEFAULT']['connection_fl']}"
    
    # ------------
    # Connect to snowflake
    with open(snow_conn_flpath) as conn_f:
        snow_conn_info = json.load(conn_f)
        sflk_session = Session.builder.configs(snow_conn_info).create()

if(sflk_session == None):
    raise(f'Unable to connect to snowflake. Validate connection information in file: {CONFIG_FL} ')

df = sflk_session.sql('select current_warehouse(), current_user(), current_role();').to_pandas()
display(df)

### Initialization

 Initialize Snowpark session


,CURRENT_WAREHOUSE(),CURRENT_USER(),CURRENT_ROLE()
0,LAB_WH,VSEKAR,DEV_BLOGGER


In [3]:
# Create database

target_db = config['DEFAULT']['db']
target_schema = config['DEFAULT']['sch']
stage = config['DEFAULT']['stage']
stmts = [
    f''' create or replace database {target_db} 
            comment='used for demonstrating snowflake and autogluon capability';  ''' 
    
    # The public schema would be created automatically by the db creation, hence the following
    # is commented out. Can be used if public schema is not desired
    # ,f''' create or replace schema {target_db}.{share_data_schema}
    #         comment='holds the base data'; '''

    ,f''' create or replace stage {target_db}.{target_schema}.{stage}
            directory = (enable=true)
            comment = 'used for holding various code & library artifacts.'; '''
    
]    
    
for stmt in stmts:
    sflk_session.sql(stmt).collect()

print(f' Created database {target_db} ')

 Created database sflk_autogluon_db 


--- 
### Closeout

    With that we are finished this section of the demo setup

In [4]:
sflk_session.close()
print('Finished!!!')

Finished!!!
